In [1]:
pip install s3fs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 257.5 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.37.34
    Uninstalling botocore-1.37.34:
      Successfully uninstalled botocore-1.37.34
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.37.34 requires botocore<1.38.0,>=1.37.34, but you have botocore 1.37.3 which is incompatible.
s3transfer 0.11.4 requires botocore<2.0a.0,>=1.37.4, but you have botocore 1.37.3 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
sft_trainer_config = '''{
    "model_name_or_path": "meta-llama/Llama-3.1-8B-Instruct",
    "training_data_path": "thesven/gsm8k-reasoning",
    "output_dir": "/mnt/output/model",
    "save_model_dir": "/mnt/output/model",
    "num_train_epochs": 1.0,
    "per_device_train_batch_size": 1,
    "per_device_eval_batch_size": 4,
    "gradient_accumulation_steps": 4,
    "gradient_checkpointing": true,
    "save_strategy": "epoch",
    "learning_rate": 1e-5,
    "weight_decay": 0.0,
    "lr_scheduler_type": "cosine",
    "include_tokens_per_second": true,
    "data_formatter_template": "### Question:{{question}} \\n\\n### Answer: {{answer}}",
    "response_template": "\\n### Answer:",
    "use_flash_attn": true,
    "fast_kernels": [true, true, true],
    "peft_method": "lora",
    "lora_post_process_for_vllm": true
}'''

In [30]:
import base64

def encode_config(config):
    base64_bytes = base64.b64encode(config.encode("ascii"))
    txt = base64_bytes.decode("ascii")
    return txt

encoded_config = encode_config(sft_trainer_config)

In [32]:
from kubeflow.training import TrainingClient
from kubernetes import client
from kubernetes.client import (
    V1EnvVar,
    V1EnvVarSource,
    V1SecretKeySelector,
    V1Volume,
    V1VolumeMount,
    V1PersistentVolumeClaimVolumeSource
)

job_name = "llama4"

tc = TrainingClient()

tc.create_job(
    job_kind="PyTorchJob",
    name=job_name,
    num_workers=1,
    num_procs_per_worker="auto",
    resources_per_worker={"gpu": 1},
    base_image="quay.io/modh/fms-hf-tuning:v2.8.2",
    env_vars=[
        V1EnvVar(name="HF_TOKEN", value_from=V1EnvVarSource(secret_key_ref=V1SecretKeySelector(key="HF_TOKEN", name="hf-token"))),
        V1EnvVar(name="SFT_TRAINER_CONFIG_JSON_ENV_VAR", value=encoded_config),
    ],
    volumes=[
        V1Volume(name="llama70",
                 persistent_volume_claim=V1PersistentVolumeClaimVolumeSource(claim_name="llama70")),
        V1Volume(name="trained-model",
                 persistent_volume_claim=V1PersistentVolumeClaimVolumeSource(claim_name="trained-model")),
    ],
    volume_mounts=[
        V1VolumeMount(name="llama70", mount_path="/mnt/model"),
        V1VolumeMount(name="trained-model", mount_path="/mnt/output"),
    ],
)

In [4]:
logs, _ = tc.get_job_logs(job_name, follow=True)

[Pod llama4-master-0]: WARNING:accelerate.commands.launch:The following values were not passed to `accelerate launch` and had defaults used instead:
[Pod llama4-master-0]: 	`--num_machines` was set to a value of `1`
[Pod llama4-master-0]: 	`--mixed_precision` was set to a value of `'no'`
[Pod llama4-master-0]: 	`--dynamo_backend` was set to a value of `'no'`
[Pod llama4-master-0]: To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[Pod llama4-master-0]: /home/tuning/.local/lib/python3.12/site-packages/fms_acceleration_moe/utils/checkpoint_utils.py:367: SyntaxWarning: invalid escape sequence '\.'
[Pod llama4-master-0]:   _reg = re.compile(f"(.*)\.({_name})\.weight")
[Pod llama4-master-0]: /home/tuning/.local/lib/python3.12/site-packages/fms_acceleration_moe/utils/checkpoint_utils.py:367: SyntaxWarning: invalid escape sequence '\.'
[Pod llama4-master-0]:   _reg = re.compile(f"(.*)\.({_name})\.weight")
[Pod llama4-master-0]: /home/tunin

In [3]:
import s3fs
import os


s3 = s3fs.S3FileSystem(
      key=os.environ["AWS_ACCESS_KEY_ID"],
      secret=os.environ["AWS_SECRET_ACCESS_KEY"],
      endpoint_url=os.environ["AWS_S3_ENDPOINT"]
   )
s3_path = os.environ["AWS_S3_BUCKET"] + "/meta-llama/Llama-3.1-8B-Instruct/adapter"
_ = s3.put("/opt/app-root/src/trained-model-1/model", s3_path, recursive=True)

In [13]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.4/680.4 kB 80.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
import os
import httpx
from openai import OpenAI


client = OpenAI(
    http_client = httpx.Client(verify=False) 
)

completion = client.chat.completions.create(
  model="llama4",
  messages=[
    {"role": "user", "content": "Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"}
  ]
)

print(completion.choices[0].message.content)

To find out how many clips Natalia sold in May, we need to calculate half of the clips she sold in April. 

In April, Natalia sold 48 clips. 
Half of 48 is 48 / 2 = 24 clips.

So, in May, Natalia sold 24 clips. 

To find the total number of clips sold in April and May, we need to add the clips sold in both months. 

48 (April) + 24 (May) = 72 clips.

Natalia sold 72 clips altogether in April and May.
